In [ ]:
import os
import scipy.io
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# --- CONFIGURATION ---
DATA_ROOT = r"D:/deep_learning/stanford-cars-dataset"
OUTPUT_DIR = "processed_data"
BUNDLE_PATH = "resnet50_inference_bundle.pth"
TEST_CSV_PATH = f"{OUTPUT_DIR}/test.csv"

# Create output folder if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Working on device: {device}")